# Día 7: Proyecto guiado - NotesApp

## Objetivo
Desarrollar una aplicación completa de notas utilizando un Smart Contract y Python con Web3.py.

## 1. Definiendo el contrato NotesApp

In [ ]:
# Definimos el contrato NotesApp en Solidity
solidity_code = '''
pragma solidity ^0.8.0;

contract NotesApp {
    // Estructura para almacenar una nota
    struct Note {
        string title;
        string content;
        uint256 timestamp;
    }
    
    // Array para almacenar todas las notas
    Note[] public notes;
    
    // Evento que se emite cuando se crea una nueva nota
    event NoteCreated(uint256 indexed id, string title, uint256 timestamp);
    
    // Función para añadir una nueva nota
    function addNote(string memory _title, string memory _content) public {
        notes.push(Note(_title, _content, block.timestamp));
        emit NoteCreated(notes.length - 1, _title, block.timestamp);
    }
    
    // Función para obtener una nota por su ID
    function getNote(uint256 _id) public view returns (string memory, string memory, uint256) {
        require(_id < notes.length, "Nota no encontrada");
        Note memory note = notes[_id];
        return (note.title, note.content, note.timestamp);
    }
    
    // Función para obtener el número total de notas
    function getNoteCount() public view returns (uint256) {
        return notes.length;
    }
}
'''

# Guardamos el código en un archivo
with open('NotesApp.sol', 'w') as file:
    file.write(solidity_code)
    
print("Contrato NotesApp guardado en 'NotesApp.sol'")

## 2. Compilando el contrato

In [ ]:
# Importamos las bibliotecas necesarias
from web3 import Web3
from solcx import compile_standard, install_solc
import json
from datetime import datetime

# Instalamos el compilador de Solidity
install_solc('0.8.0')

# Compilamos el contrato
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"NotesApp.sol": {"content": solidity_code}},
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode"]}
            }
        },
    },
    solc_version="0.8.0",
)

# Extraemos el bytecode y el ABI
bytecode = compiled_sol["contracts"]["NotesApp.sol"]["NotesApp"]["evm"]["bytecode"]["object"]
abi = compiled_sol["contracts"]["NotesApp.sol"]["NotesApp"]["abi"]

# Guardamos el ABI para uso futuro
with open('NotesApp_abi.json', 'w') as file:
    json.dump(abi, file)
    
print("Contrato compilado correctamente")

## 3. Desplegando el contrato

In [ ]:
# Conectamos con Ganache
web3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
print(f"¿Conectado a Ganache? {web3.is_connected()}")

# Obtenemos la cuenta que usaremos para el despliegue
cuenta = web3.eth.accounts[0]
print(f"Cuenta de despliegue: {cuenta}")

# Creamos el contrato
NotesApp = web3.eth.contract(abi=abi, bytecode=bytecode)

# Desplegamos el contrato
tx_hash = NotesApp.constructor().transact({'from': cuenta})
print(f"Transacción enviada: {tx_hash.hex()}")

# Esperamos a que la transacción sea minada
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
contract_address = tx_receipt.contractAddress
print(f"Contrato desplegado en: {contract_address}")

# Guardamos la dirección del contrato
with open('NotesApp_address.txt', 'w') as file:
    file.write(contract_address)

# Creamos una instancia del contrato desplegado
notes_app = web3.eth.contract(address=contract_address, abi=abi)

## 4. Interactuando con la aplicación de notas

In [ ]:
# Añadimos algunas notas
tx_hash = notes_app.functions.addNote(
    "Clase de Smart Contracts", 
    "Hoy aprendí a crear contratos inteligentes con Python y Web3.py"
).transact({'from': cuenta})
web3.eth.wait_for_transaction_receipt(tx_hash)
print("Nota 1 añadida")

tx_hash = notes_app.functions.addNote(
    "Conceptos importantes", 
    "ABI, bytecode, transacciones, eventos, gas"
).transact({'from': cuenta})
web3.eth.wait_for_transaction_receipt(tx_hash)
print("Nota 2 añadida")

# Verificamos el número de notas
note_count = notes_app.functions.getNoteCount().call()
print(f"Número total de notas: {note_count}")

## 5. Recuperando y mostrando las notas

In [ ]:
# Función para convertir timestamp a fecha legible
def timestamp_to_date(timestamp):
    return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

# Recuperamos y mostramos todas las notas
print("\n=== NOTAS GUARDADAS ===\n")
note_count = notes_app.functions.getNoteCount().call()

for i in range(note_count):
    title, content, timestamp = notes_app.functions.getNote(i).call()
    print(f"Nota #{i+1}")
    print(f"Título: {title}")
    print(f"Contenido: {content}")
    print(f"Fecha: {timestamp_to_date(timestamp)}")
    print("-" * 30)

## 6. Capturando eventos de creación de notas

In [ ]:
# Añadimos una nueva nota
tx_hash = notes_app.functions.addNote(
    "Proyecto final", 
    "Completé el proyecto NotesApp usando Smart Contracts"
).transact({'from': cuenta})
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)

# Capturamos el evento emitido
log_entries = notes_app.events.NoteCreated().process_receipt(tx_receipt)
event = log_entries[0]

print("\n=== EVENTO DE CREACIÓN DE NOTA ===\n")
print(f"ID de la nota: {event['args']['id']}")
print(f"Título: {event['args']['title']}")
print(f"Timestamp: {timestamp_to_date(event['args']['timestamp'])}")
print(f"Bloque: {event['blockNumber']}")
print(f"Transacción: {event['transactionHash'].hex()}")

## 7. Creando una interfaz simple para la aplicación

In [ ]:
def notes_app_cli():
    """Interfaz de línea de comandos simple para NotesApp"""
    print("\n===== NotesApp =====\n")
    print("1. Ver todas las notas")
    print("2. Añadir una nueva nota")
    print("3. Salir")
    
    opcion = input("\nSeleccione una opción (1-3): ")
    
    if opcion == "1":
        # Ver todas las notas
        note_count = notes_app.functions.getNoteCount().call()
        if note_count == 0:
            print("\nNo hay notas guardadas.")
        else:
            print(f"\nHay {note_count} notas guardadas:\n")
            for i in range(note_count):
                title, content, timestamp = notes_app.functions.getNote(i).call()
                print(f"Nota #{i+1}")
                print(f"Título: {title}")
                print(f"Contenido: {content}")
                print(f"Fecha: {timestamp_to_date(timestamp)}")
                print("-" * 30)
    
    elif opcion == "2":
        # Añadir una nueva nota
        title = input("\nTítulo de la nota: ")
        content = input("Contenido de la nota: ")
        
        tx_hash = notes_app.functions.addNote(title, content).transact({'from': cuenta})
        tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
        
        print("\nNota añadida correctamente.")
        print(f"Transacción: {tx_hash.hex()}")
    
    elif opcion == "3":
        print("\n¡Hasta pronto!")
        return False
    
    else:
        print("\nOpción no válida. Intente de nuevo.")
    
    return True

# Ejecutamos la interfaz una vez para demostración
# En una aplicación real, esto estaría en un bucle
notes_app_cli()

## 8. Evaluación de conocimientos

### Preguntas clave:

1. **¿Cuál es la diferencia entre .call() y .transact()?**
   - `.call()`: Lee datos del contrato sin modificar el estado de la blockchain. No consume gas y es instantáneo.
   - `.transact()`: Modifica el estado de la blockchain, consume gas y requiere ser minada.

2. **¿Qué es el ABI?**
   - El ABI (Application Binary Interface) es la interfaz que define cómo interactuar con el contrato desde fuera de la blockchain.
   - Describe las funciones, eventos y estructuras del contrato en formato JSON.
   - Es esencial para que aplicaciones externas puedan comunicarse con el contrato.

3. **¿Por qué los contratos son inmutables?**
   - Una vez desplegados, el código de los contratos no puede ser modificado.
   - Esto garantiza la confianza en el contrato, ya que las reglas no pueden cambiar.
   - Para actualizar un contrato, generalmente se despliega uno nuevo y se migran los datos.
   - La inmutabilidad es una característica fundamental de seguridad en blockchain.

## Lo aprendido en el proyecto

- Desarrollamos un contrato completo con estructuras de datos y eventos
- Implementamos funciones para añadir y recuperar notas
- Desplegamos el contrato en una blockchain local (Ganache)
- Interactuamos con el contrato desde Python usando Web3.py
- Capturamos y procesamos eventos emitidos por el contrato
- Creamos una interfaz simple para la aplicación
- Aplicamos todos los conceptos aprendidos durante la semana